# Cross-validation

With the code developed so far, it is possible to train an ANN and provide an estimate of the results it would offer in its real execution (with unseen patterns, represented by a test set). However, in this last aspect there are two factors to consider, as a consequence of the non-deterministic nature of the process we are following:

- The partitioning of the set of patterns into training/test is random (hold out), and is therefore overly dependent on good or bad luck in choosing training and test patterns.
- ANN training is not deterministic, as the initialisation of the weights is random. As before, it is too dependent on good or bad luck to start the training at a good or bad starting point.

For these two reasons, the test result of a single training is not significant when assessing the goodness of fit of the model in the presence of unseen patterns. To solve this problem, the experiment is repeated several times and the results are averaged. This can be implemented in a simple way by means of a loop; however, it is necessary to do this in an orderly way as there are two different sources of randomness.

Firstly, to minimise the randomness due to the partitioning of the data set, it is necessary to have a method that ensures that each data is used for training at least once, and for testing at least once. The most commonly used method is cross-validation. In this method, the data set is split into k disjoint subsets and k experiments are performed. In the k-th experiment, the k subset is separated for testing, and the remaining k-1 substes are used for training, performing a k-fold cross-validation. A common value is k=10, which gives a 10-fold cross-validation. Finally, the test value corresponding to the appropriate metric will be the average value of the values of the k experiments.

A widely used variant of cross-validation is stratified cross-validation. In this case, each subset is created in such a way as to keep the proportion of patterns of each class the same (or similar) as in the original dataset. This is particularly used when the data set is imbalanced.

It is usual to save not only the mean, but also the k values, in order to subsequently perform a paired hypothesis test with another model. To do this, it is necessary that both models have been trained using the same training and test sets.

This way of evaluating the model is often considered to be slightly pessimistic, i.e. the results obtained in tests are slightly worse than those that would be obtained from real training with all available data. In a hold out experiment, as mentioned above, several data are separated for testing. This means that the model is trained with less data than is available, and that by chance the data separated for testing can be of great importance (especially if there is little data). For this reason, when training with less data and possibly no "important" data, hold out is considered a pessimistic assessment. In the same way, cross-validation also separates data for testing, so it does not train on all available data, and is therefore also pessimistic. However, it is guaranteed that all data are used at least once in training and once in testing, thus trying to minimise the impact of chance in separating data, so it is considered only a slightly pessimistic evaluation.

Doing this is as simple as splitting the data set and performing a loop with k iterations in which at the k-th iteration a model is trained and evaluated with the corresponding sets. However, if the model is not deterministic, the result obtained at the k-th iteration will not be meaningful, since it is again dependent on chance. In this case, what needs to be done is a second nested loop within iteration k in which the model is repeatedly trained, and finally an average of the results is made to finally output the result of iteration k. The number of trainings must be high for the average results to be really significant, at least 50 trainings.

### Question

If this second loop is performed with a deterministic model, what will be the standard deviation of the test results obtained? Is there a difference between performing this second loop and averaging the results, or doing a single training?

`If the model is deterministic the deviation will be 0. In this case we could just do a single training iteration.`

In this way, it is possible to evaluate a model together with its hyperparameters in solving a problem. A very common situation is to compare several models (or the same model with different hyperparameters), for which this scheme has to be applied with an important caveat: the sets used in the cross-validation must be the same for each model. Since the distribution of patterns in different sets is random, having the same subsets in different runs is achieved by setting the random seed at the beginning of the program to be executed. Setting the random seed not only allows the same subsets to be generated, but is also important in order to be able to repeat the results in different runs.

It is also important to bear in mind that this methodology allows estimating the real performance of a model (although slightly pessimistic). The final model that would be used in production would be the result of training it with all the available patterns, since, as seen in the theory class, and very generally speaking, the more patterns you train with, the better the model will be.

In this assignment, you are asked to:

1. Develop a function called `crossvalidation` that receives a value `N` (equal to the number of patterns), and a value `k` (number of subsets into which the dataset is to be split), and returns a vector of length N, where each element indicates in which subset that pattern should be included.

    To do this function, one possibility is to perform the following steps:
    
    - Create a vetor with k sorted elements, from 1 to k.
    - Create a new vector with repetitions of the previous vector until its length is greater than or equal to N. The functions `repeat` and `ceil` can be used for this purpose.
    - Take the first N values of this vector.
    - Shuffle this vector (using the function `shuffle!` and return it. To use this function, the module `Random` should be loaded.
    
    No loop function should be used in the developed function.

In [9]:
using Random

function crossvalidation(N::Int64, k::Int64)
    indices = repeat(1:k, Int64(ceil(N/k)))
    indices = indices[1:N]
    shuffle!(indices)
    return indices
end;

# perform 5-fold cross-validation for a 100-elements set
crossvalidation(100, 5)

100-element Vector{Int64}:
 2
 5
 4
 1
 4
 4
 5
 5
 5
 1
 1
 4
 1
 ⋮
 5
 4
 2
 5
 2
 4
 5
 1
 2
 2
 3
 3

2. Create a new function called `crossvalidation`, which in this case receives as first argument `targets` of type `AbstractArray{Bool,2}` with the desired outputs, and as second argument a value `k` (number of subsets in which the dataset will be split), and returns a vector of length N (equal to the number of rows of targets), where each element indicates in which subset that pattern must be included. This partition has also to be stratified. To do this, the following steps can be followed:

    - Create a vector of indices, with as many values as rows in the `target` matrix.
    - Write a loop that iterates over the classes (columns in the `target` matrix), and does the following:
        - Take the number of elements belonging to that class. This can be done by making a call to the `sum` function applied to the corresponding column.
        - Make a call to the `crossvalidation` function developed earlier passing as parameters this number of elements and the value of k.
        - Update the index vector positions indicated by the corresponding column of the `targets` matrix with the values of the vector resulting from the call to the `crossvalidation` function.
        
        ### Question
        
        Could you perform these 3 operations in a single function call?
        
        ```Yes, as shown in the function we developed```
        </br>
    - Return the vector of indices.
    
    As it can be seen in this explanation, a loop iterating all classes can be used in this function. However, you need to make sure that each class has at least k patterns. A usual value is k=10. Therefore, it is important to make sure that you have at least 10 patterns of each class.
        
    ### Question
    
    What would happen if any class has a number of patterns less than k? What would be the consequences for calculating metrics?
    
    ```It could result into a case of imbalance, where some folds don't feature all the classes```
    
    > If, for whatever reason, it is impossible to ensure that you have at least 10 patterns of each class, one possibility would be to lower the value of k. In this case, consult with the teacher to assess this option, and what impact it might have on the final result of the trained models. In this case, consult with the teacher to assess this option, and what impact it might have on the final result of the trained models.

In [15]:
using DelimitedFiles 

dataset = readdlm("iris.data",',');

inputs = dataset[:,1:4];
targets = dataset[:,5];

150×3 Matrix{Bool}:
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 ⋮     
 0  0  1
 0  0  1
 0  0  1
 0  0  1
 0  0  1
 0  0  1
 0  0  1
 0  0  1
 0  0  1
 0  0  1
 0  0  1
 0  0  1

In [57]:
## Implementation of stratified crossvalidation.
##   Both the test and train sets must have the
##   same proportion of the different classes and
##   therefore improve the quality of the obtained model.
##   
##   I.E: If our dataset has 100 elements from which 80 are
##   of class a, 10 of class b and 10 of class c then our
##   10-fold crossvalidation sets must be of 10 elements from
##   which 8 are of class a, 1 of class b and 1 of class c.

## Implementacion

##   crossvalidamos los elementos
##   relativos a sus indices
##   ejemplo
##   a b b b b d d a
##    step1: 1 - - - - - - 2
##    step2: 1 1 2 1 2 - - 2
##    step3: 1 1 2 1 2 1 2 2
##   asi quedan los k-folds estratificados

function crossvalidation(targets::AbstractArray{Bool,2}, k::Int64)
    # compute the nubmer of elements in our targets dataset
    n_rows = size(targets,1)
    
    indexes = zeros(Int64, size(targets,1))
    for class in eachcol(targets)
        n_elements = sum(class)
        current_class_indexes = crossvalidation(n_elements, k)
        
        i = 1
        j = 1
        for element in class
            if element == true 
                indexes[i] = current_class_indexes[j]
                j+=1
            end
            i+=1
        end
    end
    return indexes;
end

crossvalidation (generic function with 2 methods)

3. Perform a final function called crossvalidation, but in this case with the first parameter `targets` of type `AbstractArray{<:Any,1}` (i.e. a vector with heterogeneous elements), the same second argument, and perform stratified cross-validation.

    In this case, the steps to follow in this function are not specified. However, they are similar to the previous one. A simple way to do it would be to call the function `oneHotEncoding` passing the vector `targets` as an argument.
    
      ### Question
      
      Could you develop this function without calling oneHotEncoding?
      
      ```The one hot encoding is needed in order to homogenize the targets, otherwise it would make no sense```

In [59]:
function oneHotEncoding(feature::AbstractArray{<:Any,1}, classes::AbstractArray{<:Any,1})
    numClasses = length(unique(classes))

    if (numClasses == 2)
        oneHot = Array{Bool,2}(undef, size(feature,1), 1)
        oneHot[:,1] .= (feature.==classes[1])
    else
        oneHot = Array{Bool,2}(undef, size(feature,1), numClasses)
        for numClass = 1:numClasses
            oneHot[:,numClass] .= (feature.==classes[numClass])
        end
    end
    return oneHot
end
function oneHotEncoding(feature::AbstractArray{<:Any,1})
    return oneHotEncoding(feature, unique(feature))
end

targets_ohc = oneHotEncoding(targets)


function crossvalidation(targets::AbstractArray{<:Any,1}, k::Int64)
    targets = oneHotEncoding(targets, unique(targets));
    
    crossValidationIndices = crossvalidation(size(targets,1), k);
    
    return crossValidationIndices;
end;



## Check that the folds are balanced
folds_dict = Dict([(1,[]),(2,[]),(3,[])])
for i = 1:length(folds_indexes)
    selected_fold = folds_indexes[i]
    selected_fold_list = folds_dict[selected_fold]
    append!(selected_fold_list, [targets_ohc[i,:]])
end

for i = 1:3
    check_dict = folds_dict[i]
    println("number of class_1 in fold ",i,": ",count(i->(i==[true, false, false]),check_dict))
    println("number of class_2 in fold ",i,": ",count(i->(i==[false, true, false]),check_dict))
    println("number of class_3 in fold ",i,": ",count(i->(i==[false, false, true]),check_dict))
end


number of class_1 in fold 1: 17
number of class_2 in fold 1: 17
number of class_3 in fold 1: 17
number of class_1 in fold 2: 17
number of class_2 in fold 2: 17
number of class_3 in fold 2: 17
number of class_1 in fold 3: 16
number of class_2 in fold 3: 16
number of class_3 in fold 3: 16


4. Integrate these functions into the code developed so far and define two functions to train ANNs following the stratified cross-validation strategy. To do this:

- First, it is necessary to set the random seed to ensure that the experiments are repeatable. This can be done with the `seed!` function of the `Random` module.
- Once the data is loaded and encoded, generate an index vector by calling the `crossvalidation` function.
- Create a function called `trainClassANN`, which receives as parameters the topology, the training set and the indices used for cross-validation. Optionally, it can receive the rest of the parameters used in previous assignments. Inside this function, the following steps may be followed:
    - Create a vector with k elements, which will contain the test results of the cross-validation process with the selected metric. If more than one metric is to be used, create one vector per metric.
    - Make a loop with k iterations (k folds) where, within each iteration, 4 matrices are created from the desired input and output matrices by means of the index vector resulting from the previous function. Namely, the desired inputs and outputs for training and test. As always, do this process of creating new matrices without loops.
    - Within this loop, add a call to generate the model with the training set, and test with the corresponding test set according to the value of k. This can be done by calling the `trainClassANN` function developed in previous assignments, passing as parameters the corresponding sets.
    - As indicated in the previous assignment, the training of ANNs is not deterministic, so that, for each iteration k of the cross-validation, it will be necessary to train several ANNs and return the average of the test results (with the selected metric or metrics) in order to have the test value corresponding to this k.
    - Furthermore, in the case of training ANNs, the training set can be split into training and validation if the ratio of patterns to be used for the validation set is greater than 0. To do this, use the `holdOut` function developed in a previous assignment.
    - Once the model has been trained (several times) on each fold, take the result and fill in the vector(s) created earlier (one for each metric).
    - Finally, provide the result of averaging the values of these vectors for each metric together with their standard deviations.
    - As a result of this call, at least the test value in the selected metric(s) should be returned. If the model is not deterministic (as is the case for the ANNs), it will be the average of the results of several trainings.
- Once this function is done, develop a second one, of the same name, so that it accepts as desired outputs a vector instead of an array, as in a previous assignment, and its operation is simply to make a call to this newly developed function.

> **Remarks**:
> - Although we have only seen how to train ANNs, in the next assignment we will use other models contained in another library (Scikit-Learn). The idea is to use the same code used for cross-validation with this global loop, changing only the line in which the model is generated.
> - Note that other Machine Learning models are deterministic, so they do not need the inner loop (whenever they are trained with the same data they return the same outputs), but only the loop for each fold.

In [61]:
using Flux
using Flux.Losses

function classifyOutputs(outputs::AbstractArray{<:Real,2}, threshold::Real=0.5)
    if size(outputs, 2) == 1
        output = dataset .>= threshold
    else
        (_,indicesMaxEachInstance) = findmax(outputs, dims=2);
        bool_outputs = falses(size(outputs));
        bool_outputs[indicesMaxEachInstance] .= true
    end
    return bool_outputs
end

function accuracy(outputs::AbstractArray{Bool,2}, targets::AbstractArray{Bool,2}) 

    if (size(targets,2)==1)
        return accuracy(outputs[:,1], targets[:,1])
    else
        classComparison = targets .== outputs
        correctClassifications = all(classComparison, dims=2)
        return mean(correctClassifications)
    end
end

function accuracy(outputs::AbstractArray{<:Real,2}, targets::AbstractArray{Bool,2}, threshold::Real=0.5)
    if (size(targets,2)==1)
        return accuracy(outputs[:,1], targets[:,1])
    else
        classified_outputs=classifyOutputs(outputs)
        return accuracy(classified_outputs, targets)
    end
end

# Function to generate classification artificial neural networks
function buildClassANN(numInputs::Int, topology::AbstractArray{<:Int,1}, numOutputs::Int;
                    transferFunctions::AbstractArray{<:Function,1}=fill(σ, length(topology))) 
    ann = Chain()
    numInputsLayer = numInputs
    for numOutputLayers = topology
        ann = Chain(ann..., Dense(numInputsLayer, numOutputLayers, σ))
        numInputsLayer = numOutputLayers
    end
    if (numOutputs == 1)
        ann = Chain(ann..., Dense(numInputsLayer, 1, σ))
    else
        ann = Chain(ann..., Dense(numInputsLayer, numOutputs, identity))
        ann = Chain(ann..., softmax)
    end
    return ann
end

# Function to train classification artificial neural networks
function trainClassANN(
        topology::AbstractArray{<:Int,1},  
        trainingDataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,2}}; 
        validationDataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,2}}= 
                    (Array{eltype(trainingDataset[1]),2}(undef,0,0), falses(0,0)), 
        testDataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,2}}= 
                    (Array{eltype(trainingDataset[1]),2}(undef,0,0), falses(0,0)), 
        transferFunctions::AbstractArray{<:Function,1}=fill(σ, length(topology)), 
        maxEpochs::Int=1000, 
        minLoss::Real=0.0, 
        learningRate::Real=0.01,  
        maxEpochsVal::Int=20, 
        showText::Bool=false)
    
    # Create ANN and loss function for classification problems
    training_inputs, training_outputs = trainingDataset
    validation_inputs, validation_outputs = validationDataset
    test_inputs, test_outputs = testDataset
    
    input_feats_size, output_classes_size = size(training_inputs,2), size(training_outputs,2)
    
    ann = buildClassANN(input_feats_size, topology, output_classes_size)
    loss(x, y) = (size(y,1) == 1) ? Losses.binarycrossentropy(ann(x),y) : Losses.crossentropy(ann(x),y)
    
    # Compute base array
    training_losses = Float64[]
    validation_losses = Float64[]
    test_losses = Float64[]
    
    # Metrics computation inner function
    
    current_epoch = 0  
    current_epoch_val = 0

    function compute_metrics()
        training_loss = loss(training_inputs', training_outputs')
        validation_loss = loss(validation_inputs', validation_outputs')
        test_loss = loss(test_inputs', test_outputs')
        
        training_ann_outputs = ann(training_inputs')
        validation_ann_outputs = ann(validation_inputs')
        test_ann_outputs = ann(test_inputs')
        
        training_accuracy = accuracy(training_ann_outputs', training_outputs)
        validation_accuracy = accuracy(validation_ann_outputs', validation_outputs)
        test_accuracy = accuracy(test_ann_outputs', test_outputs)

        if showText
            println("Epoch ", current_epoch)
            println("Training loss: ", training_loss, ", Training accuracy: ", 100*training_accuracy," %")
            if length(validation_inputs) > 1
                println("Validation loss: ", validation_loss, ", Validation accuracy: ", 100*validation_accuracy, " %") 
            end
            if length(test_inputs) > 1
                println("Test loss: ", test_loss, ", Test accuracy: ", 100*test_accuracy," %")
            end
        end
        return (training_loss, training_accuracy, validation_loss, 
            validation_accuracy, test_loss, test_accuracy)
    end
    
    
    # Compute initial metrics
    (training_loss, training_accuracy, validation_loss, 
        validation_accuracy, test_loss, test_accuracy) = compute_metrics()
    push!(training_losses, training_loss)
    push!(validation_losses, validation_loss)
    push!(test_losses, test_loss)
    
        
    # Store initial ANN as the 'best'
    best_validation_loss = validation_loss;
    final_ann = deepcopy(ann);
    
    # Training loop
    while (current_epoch < maxEpochs) && (training_loss > minLoss) && (current_epoch_val < maxEpochsVal)
        current_epoch += 1
        
        Flux.train!(loss, Flux.params(ann), [(training_inputs', training_outputs')], ADAM(learningRate))
        (training_loss, training_accuracy, validation_loss, 
            validation_accuracy, test_loss, test_accuracy) = compute_metrics();
        
        push!(training_losses, training_loss)
        push!(validation_losses, validation_loss)
        push!(test_losses, test_loss)
        
        # Check for early stop (only if we have a validation dataset)
        if length(validation_inputs) > 1
            if (validation_loss < best_validation_loss)
                # reset the number of validation epochs, because we have an improved metric
                # and store current ann as best
                if showText
                    println("[->] Found new best model: old_val_loss=",validation_loss,", new_val_loss=",best_validation_loss)
                end
                current_epoch_val = 0;
                best_validation_loss = validation_loss;
                final_ann = deepcopy(ann);
            else
                current_epoch_val += 1;
            end
        end
        
    end
    
    return (final_ann, training_losses, validation_losses, test_losses)
end



trainClassANN (generic function with 2 methods)

In [72]:
using Dates;

function holdOut(N::Int, P::Real)    
    # generate random index vector
    index_vector=Random.randperm(MersenneTwister(Dates.datetime2epochms(Dates.now())), N)
    # if we are holding out 0, return the idx vector
    if P==0
        return index_vector
    end
    # compute the cut vector
    cut_set = index_vector[1:cut_point]
    train_set = index_vector[cut_point:length(index_vector)]
    return train_set, cut_set
end

holdOut(10,0)

10-element Vector{Int64}:
  5
  1
  7
  4
  3
 10
  6
  8
  2
  9

In [62]:
## Training with k-fold cross-validation
##   1) divide the dataset into k-folds of train/test sets
##   2) for each fold, create a validation set making a hold-out into the train set
##   3) train the ann k times and for each training check the desired metrics and store them
##   4) once the k-trains is finished, average the metrics vectors

function holdOut(N::Int, P::Real)
    index_vector = [1:N;]'
    cut_point = floor(Int,N*P)
    index_vector=Random.randperm(MersenneTwister(Dates.datetime2epochms(Dates.now())), N)
    
    # return (indexes_training, indexes_texting)
    cut_set = index_vector[1:cut_point]
    train_set = index_vector[cut_point:length(index_vector)]
    return train_set, cut_set
end





function trainClassANN(
        topology::AbstractArray{<:Int,1}, 
        trainingDataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,2}}, 
        kFoldIndices::	Array{Int64,1}; 
        transferFunctions::AbstractArray{<:Function,1}=fill(σ, length(topology)), 
        maxEpochs::Int=1000, 
        minLoss::Real=0.0, 
        learningRate::Real=0.01, 
        repetitionsTraining::Int=1, 
        validationRatio::Real=0.0, 
        maxEpochsVal::Int=20)

    
    # Transform the kFoldIndexes into a set to remove duplicate and
    # compute the number of folds as the size of that set
    numFolds = size(unique(kFoldIndices))[1]
    
    # create our metrics vectors
    train_losses_array = []
    train_accuracy_array = []
    
    test_losses_array  = []
    test_accuracy_array = []
    
    validation_losses_array = []
    validation_accuracy_array = []
        
    for numFold = 1:numFolds
        # Extract the train and test dataset using the k-folds
        train_in  = trainingDataset[1][kFoldIndices.!=numFold,:]
        train_out = trainingDataset[2][kFoldIndices.!=numFold,:]
        
        test_in   = trainingDataset[1][kFoldIndices.==numFold,:]
        test_out  = trainingDataset[2][kFoldIndices.==numFold,:]
        
        # If we are using a validation dataset, perform a holdout
        train_idx, val_idx = holdOut(size(train_in,1), validationRatio)
            
        train_dataset      = train_in[train_idx,:], train_out[train_idx,:]
        validation_dataset = train_in[val_idx,:], train_out[val_idx,:]
        test_dataset       = test_in, test_out

        for i=1:repetitionsTraining
            (final_ann, train_losses, validation_losses, test_losses, training_accuracies, 
                validation_accuracies, test_accuracies) = trainClassANN(topology, 
                                                                        train_dataset, 
                                                                        validationDataset=validation_dataset, 
                                                                        testDataset=test_dataset, 
                                                                        transferFunctions=transferFunctions, 
                                                                        maxEpochs=maxEpochs, 
                                                                        minLoss=minLoss, 
                                                                        learningRate=learningRate, 
                                                                        maxEpochsVal=maxEpochsVal,
                                                                        showText=true)

            # Append the data to our arrays
            append!(train_losses_array, train_losses)
            append!(train_accuracy_array, training_accuracies)

            append!(test_losses_array, test_losses)
            append!(test_accuracy_array, test_accuracies)

            append!(validation_losses_array, validation_losses)
            append!(validation_accuracy_array, validation_accuracies)
        end
    end
                
    return (train_losses_array, train_accuracy_array, test_losses_array, test_accuracy_array,
        validation_losses_array, validation_accuracy_array)
end

trainClassANN (generic function with 2 methods)

In [8]:
function trainClassANN(topology::AbstractArray{<:Int,1},
        trainingDataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,1}},
        kFoldIndices::	Array{Int64,1};
        transferFunctions::AbstractArray{<:Function,1}=fill(σ, length(topology)),
        maxEpochs::Int=1000, minLoss::Real=0.0, learningRate::Real=0.01,repetitionsTraining::Int=1, 
        validationRatio::Real=0.0, maxEpochsVal::Int=20)
    
    training_inputs, training_outputs = trainingDataset
    
    return trainClassANN(topology, (training_inputs, oneHotEncoding(training_outputs)),
    kFoldIndices, transferFunctions, maxEpochs, minLoss, learningRate, repetitionsTraining,
    validationRatio, maxEpochsVal)
end

trainClassANN (generic function with 2 methods)